<img src="http://hilpisch.com/images/tr_logo_long.png" width="40%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# Eikon Data API

**Calibrating an Options Pricing Model**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial covers

* Retrieving Options Data
* Merton (1976) Jump-Diffusion Model
* Calibration of the Model

## Imports and Versions

The following imports several **packages** as used in the following.

In [ ]:
import math
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import scipy  # Scientific Python
from scipy.integrate import quad  # Numerical Integration
import scipy.optimize as spo  # Optimization
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [ ]:
import sys
print(sys.version)

In [ ]:
ek.__version__

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
cf.__version__

In [ ]:
scipy.__version__

## Connecting to Eikon Data API

This code sets the `app_id` to connect to the **Eikon Data API Proxy** which needs to be running locally. It requires the previously created text file `eikon.cfg` to be in the current working directory.

In [ ]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location

In [ ]:
ek.set_app_id(cfg['eikon']['app_id'])

## Retrieving Options Data

### The Raw Data 

In what follows, data is retrieved that comprises fields for the **option type (put or call), the strike price, the closing price and the implied volatility**.

In [ ]:
fields = ['CF_DATE', 'EXPIR_DATE', 'PUTCALLIND', 'STRIKE_PRC', 'CF_CLOSE', 'IMP_VOLT']

Using the `ek.get_data()` method, allows data retrieval for a **single maturity of index options and multiple data fields** at the same time.

In [ ]:
dax = ek.get_data('0#GDAXM8*.EX', fields=fields)[0]

In [ ]:
len(dax)

In [ ]:
dax.head(15)

Let us pick and store the closing value for the index.

In [ ]:
GDAXI = dax.iloc[0]['CF_CLOSE']

In [ ]:
GDAXI

In the following, we work with **call options** related data only.

In [ ]:
calls = dax[dax['PUTCALLIND'] == 'CALL'].copy()

Dates delivered as `str` objects need to be transformed to `pd.Timestamp` objects.

In [ ]:
for col in ['CF_DATE', 'EXPIR_DATE']:
    calls[col] = calls[col].apply(lambda date: pd.Timestamp(date))

In [ ]:
calls.info()

It might make sense, to restrict the options data to those options that are not too far in or out of the money.

In [ ]:
limit = 500

In [ ]:
calls = calls[abs(calls['STRIKE_PRC'] - GDAXI) < limit]

In [ ]:
calls.info()

The **options chain sub set** relevant for the model calibration to follow.

In [ ]:
calls

### The Data Visualized

The separation allows for easily **visualization** of the closing prices and the implied volatilities.

In [ ]:
calls.set_index('STRIKE_PRC')[['CF_CLOSE', 'IMP_VOLT']].iplot(subplots=True,
                                                             mode='lines+markers',
                                                            symbol='dot', size=6)

## Merton (1976) Jump-Diffusion Model

In the Merton (1976) jump-diffusion model, the **risk-neutral index level dynamics** are given by the SDE

$$
dS_{t}=(r-r_{J})S_{t}dt+\sigma S_{t}dZ_{t}+J_{t}S_{t}dN_{t}
$$

The variables and parameters have the following meaning:

* $S_{t}$ index level at date $t$
* $r$ constant risk-less short rate 
* $r_{J}\equiv \lambda \cdot \left(e^{\mu_{J}+\delta^{2}/2}-1\right)$ drift correction for jump
* $\sigma$ constant volatility of $S$
* $Z_{t}$ standard Brownian motion
* $J_{t}$ jump at date $t$ with distribution $\log(1+J_{t}) \approx \mathbf{N}\left(\log(1+\mu_{J})-\frac{\delta^{2}}{2},\delta^{2}\right)$
* $\mathbf{N}$ as the cumulative distribution function of a standard normal random variable
* $N_{t}$ Poisson process with intensity $\lambda$

For details, refer to **Hilpisch, Yves (2015): _Derivatives Analytics with Python._ Wiley Finance.**

The **characteristic function for the Merton (1976) model** is given as:

$$
\varphi_{0}^{M76}(u,T)=\exp\left(\left(iu\omega -\frac{u^{2}\sigma^{2}}{2}+\lambda \left(e^{iu\mu_{J}-u^{2}\delta^{2}/2}-1\right)\right)T\right)
$$

where the **risk-neutral drift term $\omega$** takes on the form

$$
\omega=r-\frac{\sigma^{2}}{2}-\lambda\left(e^{\mu_{J}+\delta^{2}/2}-1\right)
$$

Combining this with the option pricing result from Lewis (2001) we get for the **price of a European call option**

$$
C_{0}=S_{0}-\frac{\sqrt{S_{0}K}e^{-rT/2}}{\pi}\int_{0}^{\infty} \mathbf{Re}\left[e^{izk} \varphi_{0}^{M76}(z-i/2,T)\right] \frac{dz}{z^2+1/4}
$$

Let us implement the **European call option pricing formula** in Python. First, the **characteristic function**.

In [ ]:
def M76_characteristic_function(u, T, r, sigma, lamb, mu, delta):
    omega = r - 0.5 * sigma ** 2 - lamb * (np.exp(mu + 0.5 * delta ** 2) - 1)
    value = np.exp((1j * u * omega - 0.5 * u ** 2 * sigma ** 2 +
            lamb * (np.exp(1j * u * mu - u ** 2 * delta ** 2 * 0.5) - 1))  * T)
    return value

Second, the **integration function**.

In [ ]:
def M76_integration_function(u, S0, K, T, r, sigma, lamb, mu, delta):
    JDCF = M76_characteristic_function(u - 0.5 * 1j, T, r,
                                       sigma, lamb, mu, delta)
    value = 1 / (u ** 2 + 0.25) * (np.exp(1j * u * math.log(S0 / K))
                                    * JDCF).real
    return value

Third, the **evaluation of the integral** via numerical quadrature.

In [ ]:
def M76_value_call_INT(S0, K, T, r, sigma, lamb, mu, delta):
    int_value = quad(lambda u: M76_integration_function(u, S0, K, T, r,
                    sigma, lamb, mu, delta), 0, 50, limit=250)[0]
    call_value = S0 - np.exp(-r * T) * math.sqrt(S0 * K) / math.pi * int_value
    return call_value

Fourth, a **numerical example**.

In [ ]:
K = GDAXI  # strike level = index level
T = 1.0  # call option maturity
r = 0.005  # constant short rate
sigma = 0.4  # constant volatility of diffusion
lamb = 1.0  # jump frequency p.a.
mu = -0.2  # expected jump size
delta = 0.1  # jump size volatility

In [ ]:
print ('Value of Call Option %8.3f' \
            % M76_value_call_INT(GDAXI, K, T, r, sigma, lamb, mu, delta))

## Calibration of the Model

In simple terms, the problem of **calibration** is to find parameters for the Merton (1976) model such that observed market quotes of liquidly traded plain vanilla options are replicated as good as possible. To this end, one defines an error function that is to be minimized. Such a function could be the Root Mean Squared Error (RMSE). The task is then to solve the problem

$$
\min_{\sigma, \lambda, \mu_{J}, \delta } \sqrt{\frac{1}{N}\sum_{n=1}^{N}\left( C_{n}^{*} - C_{n}^{M76}(\sigma, \lambda, \mu_{J}, \delta )\right)^{2}}
$$
with the $C_{n}^{*}$ being the market or input prices and the $C_{n}^{M76}$ being the model or output prices for the options $n=1,...,N$.

Next, we define an **error function** in Python for the calibration. 

In [ ]:
i = 0; min_RMSE = 100.
def M76_error_function(p0):
    global i, min_RMSE
    sigma, lamb, mu, delta = p0
    if sigma < 0.0 or delta < 0.0 or lamb < 0.0:
        return 500.0
    se = []
    for row, option in calls.iterrows():
        T = (option['EXPIR_DATE'] - option['CF_DATE']).days / 365.
        model_value = M76_value_call_INT(GDAXI, option['STRIKE_PRC'], T,
                                         r, sigma, lamb, mu, delta)
        se.append((model_value - option['CF_CLOSE']) ** 2)
    RMSE = math.sqrt(sum(se) / len(se))
    min_RMSE = min(min_RMSE, RMSE)
    if i % 100 == 0:
        print ('%4d |' % i, np.array(p0), '| %7.3f | %7.3f' % (RMSE, min_RMSE))
    i += 1
    return RMSE

The calibration is done in two steps. First, a **global optimization**.

In [ ]:
%%time
import scipy.optimize as sop
np.set_printoptions(suppress=True,
                    formatter={'all': lambda x: '%6.3f' % x})
p0 = sop.brute(M76_error_function, ((0.10, 0.201, 0.025),
                   (0.10, 0.81, 0.20), (-0.40, 0.01, 0.10),
                   (0.00, 0.126, 0.025)), finish=None)

Second, the **local (convex) optimization**.

In [ ]:
%%time
opt = sop.fmin(M76_error_function, p0, xtol=0.00001,
                    ftol=0.00001, maxiter=750, maxfun=1500)

Given the **optimal parameters**, the model prices after calibration can be calculated.

In [ ]:
sigma, lamb, mu, delta = opt
calls['MODEL_PRICE'] = 0.0
for row, option in calls.iterrows():
    T = (option['EXPIR_DATE'] - option['CF_DATE']).days / 365.
    calls.loc[row, 'MODEL_PRICE'] = M76_value_call_INT(GDAXI, option['STRIKE_PRC'],
                                T, r, sigma, lamb, mu, delta)

In [ ]:
calls

The calibration yields **model prices** not "too different" from the market prices.

In [ ]:
calls[['CF_CLOSE', 'MODEL_PRICE']].iplot(mode='lines+markers', size=6.0)

The following calculates and visualizes the **absolute and relative pricing errors in percent**.

In [ ]:
diffs = calls['MODEL_PRICE'] - calls['CF_CLOSE']

In [ ]:
diffs.iplot(kind='bar', yTitle='pricing errors [EUR]')

In [ ]:
rdiffs = (calls['MODEL_PRICE'] - calls['CF_CLOSE']) / calls['CF_CLOSE'] * 100

In [ ]:
rdiffs.iplot(kind='bar', yTitle='pricing errors [%]')

## Conclusions

This tutorial covers:

* Retrieving Options Data
* Merton (1976) Jump-Diffusion Model
* Calibration of the Model

## References

For financial, numerical and programming details with regard to option pricing models and their calibration in Python, refer to:

* Hilpisch, Yves (2015): _Derivatives Analytics with Python._ Wiley Finance.

## Eikon Data API Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Eikon Search Bar.

* [Article on Chains](https://developers.thomsonreuters.com/article/simple-chain-objects-ema-part-1)

<img src="http://hilpisch.com/images/tr_logo_long.png" width="40%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">